In [12]:
### Import:

In [13]:
import pandas as pd
import numpy as np


### 1. Caricamento del dataset

In [14]:
try:
    df = pd.read_csv("./data/monsters.csv")
    print("✅ Dataset caricato con successo.")
except FileNotFoundError:
    print("❌ Errore: File non trovato.")
    raise


✅ Dataset caricato con successo.


### --- 2. GESTIONE DEI TIRI SALVEZZA ---

In [15]:
stats_map = {
    'strength': 'strength_save',
    'dexterity': 'dexterity_save',
    'constitution': 'constitution_save',
    'intelligence': 'intelligence_save',
    'wisdom': 'wisdom_save',
    'charisma': 'charisma_save'
}

print("--- Elaborazione Tiri Salvezza ---")

for stat_col, save_col in stats_map.items():
    if stat_col in df.columns and save_col in df.columns:
        modifiers = np.floor_divide(df[stat_col] - 10, 2)
        condition = df[save_col].isnull()
        df[save_col] = np.where(condition, modifiers, df[save_col])
        
        if condition.sum() > 0:
            print(f"✅ {save_col}: Imputati {condition.sum()} valori mancanti.")


--- Elaborazione Tiri Salvezza ---


### --- 3. GESTIONE PERCEPTION ---

In [16]:
print("--- Elaborazione Perception ---")
if 'wisdom' in df.columns and 'perception' in df.columns:
    wis_mod = np.floor_divide(df['wisdom'] - 10, 2)
    condition = df['perception'].isnull()
    df['perception'] = np.where(condition, wis_mod, df['perception'])
    print(f"✅ Perception: Imputati {condition.sum()} valori mancanti.")


--- Elaborazione Perception ---
✅ Perception: Imputati 0 valori mancanti.


### --- 4. RIMOZIONE COLONNE RIDONDANTI ---

In [17]:
cols_to_drop = ['slug', 'armor_desc']
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]

if existing_cols_to_drop:
    df = df.drop(columns=existing_cols_to_drop)
    print(f"\n✅ Colonne rimosse: {existing_cols_to_drop}")


### --- 5. TRASFORMAZIONE HIT DICE ---

In [18]:
print("--- Elaborazione Hit Dice ---")
if 'hit_dice' in df.columns:
    hit_dice_clean = df['hit_dice'].fillna('0d0').astype(str)
    extracted_values = hit_dice_clean.str.split('d', expand=True)[0]
    numeric_values = pd.to_numeric(extracted_values, errors='coerce')
    clean_values = np.nan_to_num(numeric_values, nan=0)
    df['hit_dice_count'] = clean_values.astype(int)
    df = df.drop(columns=['hit_dice'])
    print("✅ Hit Dice convertiti.")
else:
    print("⚠️ Colonna 'hit_dice' non trovata.")


--- Elaborazione Hit Dice ---
⚠️ Colonna 'hit_dice' non trovata.


### --- 6. TRASFORMAZIONE SIZE ---

In [19]:
print("--- Elaborazione Size ---")
if 'size' in df.columns:
    size_mapping = {'Tiny': 1, 'Small': 2, 'Medium': 3, 'Large': 4, 'Huge': 5, 'Gargantuan': 6, 'Titanic': 7}
    df['size_numeric'] = df['size'].map(size_mapping).fillna(3).astype(int)
    df = df.drop(columns=['size'])
    print("✅ Size convertita in scala ordinale (1-7).")


--- Elaborazione Size ---


### --- 7. ELABORAZIONE DIFESE (Sovrascrittura In-Place) ---
Contiamo i valori e sovrascriviamo le colonne originali per mantenerne nome e posizione.

In [20]:
print("--- Elaborazione Difese (In-Place Count) ---")

def count_items(text):
    if pd.isna(text) or str(text).strip() == '':
        return 0
    # Normalizzazione e split
    text = str(text).lower().replace(';', ',').replace(' and ', ',')
    items = [x.strip() for x in text.split(',') if x.strip()]
    return len(items)

defense_cols = ['damage_vulnerabilities', 'damage_resistances', 'damage_immunities', 'condition_immunities']

for col in defense_cols:
    if col in df.columns:
        # Calcolo conteggi
        counts = df[col].apply(count_items)
        
        # Gestione vulnerabilità (valori negativi)
        if col == 'damage_vulnerabilities':
            counts = counts * -1
            
        # Sovrascrittura diretta: preserva il nome e la posizione della colonna
        df[col] = counts.astype(int)
        
        print(f"✅ {col}: Convertita in numerico (range: {df[col].min()} a {df[col].max()}).")


--- Elaborazione Difese (In-Place Count) ---
✅ damage_vulnerabilities: Convertita in numerico (range: -2 a 0).
✅ damage_resistances: Convertita in numerico (range: 0 a 9).
✅ damage_immunities: Convertita in numerico (range: 0 a 10).
✅ condition_immunities: Convertita in numerico (range: 0 a 12).


### --- RIORDINAMENTO GENERALE ---
Gestiamo solo hit_dice e size che sono stati rinominati/creati da zero.
Le colonne delle difese sono rimaste al loro posto originale.

In [21]:
cols = df.columns.tolist()

def move_col(df_cols, col_to_move, after_col):
    if col_to_move in df_cols:
        df_cols.remove(col_to_move)
        if after_col in df_cols:
            idx = df_cols.index(after_col)
            df_cols.insert(idx + 1, col_to_move)
        else:
            df_cols.append(col_to_move)
    return df_cols

cols = move_col(cols, 'hit_dice_count', 'hit_points')
cols = move_col(cols, 'size_numeric', 'name')

df = df[cols]


### --- 8. SALVATAGGIO ---

In [22]:
output_path = "./data/monsters.csv"
try:
    df.to_csv(output_path, index=False)
    print(f"\n💾 Salvataggio completato! File aggiornato: {output_path}")
    # Preview finale delle colonne modificate
    preview_cols = ['name'] + defense_cols
    print(df[preview_cols].head())
except:
    df.to_csv("monsters_processed.csv", index=False)
    print(f"\n💾 File salvato come: monsters_processed.csv")



💾 Salvataggio completato! File aggiornato: ./data/monsters.csv
                 name  damage_vulnerabilities  damage_resistances  \
0             Aboleth                       0                   0   
1             Acolyte                       0                   0   
2  Adult Black Dragon                       0                   0   
3   Adult Blue Dragon                       0                   0   
4  Adult Brass Dragon                       0                   0   

   damage_immunities  condition_immunities  
0                  0                     0  
1                  0                     0  
2                  1                     0  
3                  1                     0  
4                  1                     0  
